In [1]:
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat

In [2]:
click_f = loadmat('data/epinions/rating.mat')['rating']
trust_f = loadmat('data/epinions/trustnetwork.mat')['trustnetwork']

In [3]:
click_f

array([[    1,     1,     3,     2],
       [    1,     2,     2,     2],
       [    1,     3,     3,     2],
       ...,
       [22166, 43538,     5,     5],
       [22166, 38711,     3,     4],
       [22166, 41790,     5,     3]])

In [4]:
trust_f

array([[15373,  9831],
       [ 4247,  9831],
       [ 4644,  9831],
       ...,
       [13181, 15645],
       [  897,  8000],
       [ 8000,   897]], dtype=uint16)

In [5]:
click_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0

In [6]:
for x in click_f:
    uid = x[0]
    iid = x[1]
    label = x[2]
    user_count = max(user_count, uid)
    item_count = max(item_count, iid)
    rate_count = max(rate_count, label)
    click_list.append([uid, iid, label])

In [7]:
pos_list = []
for x in click_list:
	pos_list.append((x[0], x[1], x[2]))

In [8]:
pos_list = list(set(pos_list))
random.shuffle(pos_list)
num_test = int(len(pos_list) * 0.1)
test_set = pos_list[:num_test]
valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[2 * num_test:]
print('Train samples: {}, Valid samples: {}, Test samples: {}'.format(len(train_set), len(valid_set), len(test_set)))

Train samples: 730054, Valid samples: 91256, Test samples: 91256


In [9]:
with open('data/dataset_epinions.pkl', 'wb') as f:
	pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(valid_set, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)

In [10]:
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
valid_df = pd.DataFrame(valid_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(test_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(click_list, columns = ['uid', 'iid', 'label'])
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')

In [11]:
train_df

,uid,iid,label
527993,1,5,3
551242,1,2,2
479205,1,8,3
457030,1,4,3
589434,1,10,3
...,...,...,...
652375,22166,41790,5
379267,22166,38711,3
188869,22166,83922,5
555759,22166,53887,5


In [12]:
for user in tqdm(range(user_count + 1)):
    user_df = train_df[train_df['uid'] == user]
    user_items = user_df['iid'].tolist()
    user_ratings = user_df['label'].tolist()
    if len(user_items) == 0:
        u_items_list.append([(0, 0)])
    else:
        u_items_list.append([(iid, rating) for iid, rating in zip(user_items, user_ratings)])

100%|██████████| 22167/22167 [00:22<00:00, 1000.28it/s]


In [13]:
for item in tqdm(range(item_count + 1)):
    item_df = train_df[train_df['iid'] == item]
    item_users = item_df['uid'].tolist()
    item_ratings = item_df['label'].tolist()
    if len(item_users) == 0:
        i_users_list.append([(0, 0)])
    else:
        i_users_list.append([(uid, rating) for uid, rating in zip(item_users, item_ratings)])

100%|██████████| 296278/296278 [04:37<00:00, 1067.85it/s]


In [14]:
for x in trust_f:
    uid = x[0]
    fid = x[1]
    if uid > user_count or fid > user_count:
        continue
    trust_list.append([uid, fid])

In [15]:
trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')

In [16]:
trust_df

,uid,fid
283312,2,7659
84889,2,829
221672,2,21911
247685,2,7336
306637,2,6339
...,...,...
285484,22164,5864
283826,22164,11427
285362,22164,10897
135466,22165,2198


In [17]:
for user in tqdm(range(user_count + 1)):
    user_df = trust_df[trust_df['uid'] == user]
    u_users = user_df['fid'].unique().tolist()
    if len(u_users) == 0:
        u_users_list.append([0])
        u_users_items_list.append([[(0, 0)]])
    else:
        u_users_list.append(u_users)
        uu_items = []
        for uid in u_users:
            uu_items.append(u_items_list[uid])
        u_users_items_list.append(uu_items)

100%|██████████| 22167/22167 [00:12<00:00, 1771.78it/s]


In [18]:
with open('data/list_epinions.pkl', 'wb') as f:
	pickle.dump(u_items_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(u_users_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(u_users_items_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump(i_users_list, f, pickle.HIGHEST_PROTOCOL)
	pickle.dump((user_count, item_count, rate_count), f, pickle.HIGHEST_PROTOCOL)